In [1]:
#这个是将多帧dicom中所有的图片进行提取
!pip install pydicom

    100% |████████████████████████████████| 2.0MB 740kB/s ta 0:00:011


In [2]:
import pydicom
import math
import os
import sys
import time
import json


def generateImagesFromDicom(dataset, filename="extracted_folder", extractfps=0):
    """
        Go through all items in the dataset and print them with custom format

        Modelled after Dataset._pretty_str()
    """
    cine_rate = 12    
    for data_element in dataset:
        if data_element.name == 'Cine Rate':
            print(f"Cine Rate:{data_element.value}, {type(data_element.value)}")
            cine_rate = data_element.value.real
        if data_element.name == 'Pixel Data':
            pixel_datas = data_element.value
       
    # check frame number and tranfersyntex here
    frmpath = filename + '_frms'
    os.makedirs(filename + '_frms', exist_ok=True)
    
    print('Extract frames from [Pixel Data]:')
    if 0 == extractfps or extractfps > cine_rate:
        delta_step = cine_rate / cine_rate
    else:
        delta_step = cine_rate / extractfps
    frmindex = 0
    for sequence_index, frm in enumerate(pydicom.encaps.generate_pixel_data_frame(pixel_datas)):
        delta = sequence_index - (frmindex * delta_step)
        if delta >= 1 or delta < 0:
            continue
        frmname = os.path.join(frmpath, f"frm-{(frmindex+1):04d}.png")
        #print("generateImagesFromDicom: {0} {1} : {2} bytes".format(">>", frmname, len(frm)))
        with open(frmname, 'wb+') as f:
            f.write(frm)
        frmindex += 1
    print(f"Total generate {frmindex} image...")

    return frmpath if frmindex > 1 else None

def record2File(imagedir, fps, raw_file_type='mp4'):
    """when finish the extract action, make a file to record the status"""
    jsonfile = "lmstart.json"
    now = time.localtime(time.time())
    datatime = "{0}-{1}-{2} {3}:{4}:{5}".format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour,now.tm_min, now.tm_sec)
    recorder = {"datetime":datatime, "fps":fps, "imagepath":imagedir, "raw_file_type": raw_file_type}
    print(recorder)
    
    with open(os.path.join(imagedir, jsonfile), "w") as fp:
        json.dump(recorder, fp, ensure_ascii=True)
    return 0
    
def extractDcmToImages(dicom_file, extractfps):
    print("dcmfile:", dicom_file)
    if os.path.isfile(dicom_file) and  os.path.splitext(dicom_file)[1] ==".dcm":
        print("dcmfile is file? :", dicom_file)
        ds = pydicom.dcmread(dicom_file)
        try:
            image_dir = generateImagesFromDicom(ds, filename=dicom_file, extractfps=extractfps)
        except Exception as e:
            print('Err:dcm library error')
            image_dir = None
        if image_dir:
            record2File(imagedir=image_dir, fps=extractfps, raw_file_type="dcm")
        return image_dir
    else:
        return None

In [6]:
def extract_all_dcm_imgs(dcm_root_dir, count=0):
    if not os.path.exists(dcm_root_dir):
        print(f"Dir not exist:{dcm_root_dir}")
        return -1
    subdirs = []
    dirs = sorted(os.listdir(dcm_root_dir))
    print(f"dirs={dirs}")
    for item in dirs:
        path = os.path.join(dcm_root_dir, item)
        if os.path.isdir(path):
            subdirs.append(path)
    if len(subdirs) <1:
        subdirs.append(dcm_root_dir)
    print(f"subdirs={subdirs}")
    
    folder_details = []
    dcm_f_count = 0
    for ct, dcm_dir in enumerate(subdirs):
        print("*"*32)
        print('Handle with {0} [{1}/]'.format(dcm_dir, ct + 1))
        files_in_dir = os.listdir(dcm_dir)
        if len(files_in_dir) < 2:
            getLogger().info(f"Empty dirctory : {dcm_dir}")
            continue
        else:
            for dcmf in files_in_dir:
                fullfilename = os.path.join(dcm_dir, dcmf)
                if not os.path.isfile(fullfilename):
                    continue
                print("file in dir:", fullfilename)
                extractDcmToImages(fullfilename, 100)
                dcm_f_count = dcm_f_count+1
                if count != 0 and count == dcm_f_count:
                    print(f"\n<<<already extract {count} directory dcm file~")
                    return
    return 0

In [7]:
## worked at 21/08/22
#dcm_root_dir=r"/mnt/d/workspace/01-videos"
#extract_all_dcm_imgs(dcm_root_dir, 1)

In [8]:
##working at 22/01/29
dcm_root_dir=r"/mnt/o/download/220222_thyroids"
extract_all_dcm_imgs(dcm_root_dir, 0)

dirs=['01.dcm', '06.dcm', '07.dcm', '08.dcm', '09.dcm', '1.3.6.1.4.1.52026.100007044.2012.8.1.1.20220208225920060_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.1.1.1.20220218161356838_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.1.2.1.20220218142848081_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.105.1.1.20220129145138521_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.105.4.1.20220129020504122_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.107.1.1.20220129191716000_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.107.2.1.20220129220327106_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.107.3.1.20220130002833040_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.107.4.1.20220129014418393_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.123.1.1.20220129235707638_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.123.2.1.20220216172138940_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.126.1.1.20220206234329588_MARK.json', '1.3.6.1.4.1.52026.66519049.5308.32.2.1.20220128210627732_MARK.json', '1.3.6.1.4.1.52026.66519049

Cine Rate:15, <class 'pydicom.valuerep.IS'>
Extract frames from [Pixel Data]:
Total generate 205 image...
{'datetime': '2022-2-22 20:59:23', 'fps': 100, 'imagepath': '/mnt/o/download/220222_thyroids/13.dcm_frms', 'raw_file_type': 'dcm'}
file in dir: /mnt/o/download/220222_thyroids/16.dcm
dcmfile: /mnt/o/download/220222_thyroids/16.dcm
dcmfile is file? : /mnt/o/download/220222_thyroids/16.dcm
Cine Rate:15, <class 'pydicom.valuerep.IS'>
Extract frames from [Pixel Data]:
Total generate 221 image...
{'datetime': '2022-2-22 20:59:25', 'fps': 100, 'imagepath': '/mnt/o/download/220222_thyroids/16.dcm_frms', 'raw_file_type': 'dcm'}
file in dir: /mnt/o/download/220222_thyroids/17.dcm
dcmfile: /mnt/o/download/220222_thyroids/17.dcm
dcmfile is file? : /mnt/o/download/220222_thyroids/17.dcm
Cine Rate:15, <class 'pydicom.valuerep.IS'>
Extract frames from [Pixel Data]:
Total generate 512 image...
{'datetime': '2022-2-22 20:59:27', 'fps': 100, 'imagepath': '/mnt/o/download/220222_thyroids/17.dcm_frms

0